In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 경로 설정
base_model_path = "Qwen/Qwen2-7B-Instruct"
adapter_path = "./qwen2-7b-rag-ko/checkpoint-285"
merged_model_path = "./output_dir"

# 디바이스 설정
device_arg = {"device_map": "auto"}

# 베이스 모델 로드
print(f"Loading base model from: {base_model_path}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)

# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
model = PeftModel.from_pretrained(base_model, adapter_path, **device_arg)
model = model.merge_and_unload()

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 저장
print(f"Saving merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("✅ 모델과 토크나이저 저장 완료")

Loading base model from: Qwen/Qwen2-7B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading and merging PEFT from: ./qwen2-7b-rag-ko/checkpoint-285
Saving merged model to: ./output_dir
✅ 모델과 토크나이저 저장 완료


In [2]:
from huggingface_hub import HfApi
api = HfApi()

username = "iamjoon"

In [3]:
MODEL_NAME = 'qwen2-7b-rag-ko-checkpoint-285'

In [ ]:
api.create_repo(
    token="hf_여러분의 키 값",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token="hf_여러분의 키 값",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir",
)

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]